In [1]:
import gym # openAi gym
from gym import envs
import numpy as np 
import datetime
import keras 
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from time import sleep

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
print("OK")

Using TensorFlow backend.


OK


In [36]:
#print(envs.registry.all())
env = gym.make('Ant-v1')
#env = gym.make('Taxi-v2')
env.reset()
env.render()

[2019-11-22 16:15:53,957] Making new env: Ant-v1


/home/kavindie/Documents/git_repo/gym/gym/envs/mujoco/assets/ant.xml
Enter


In [37]:
# Let's first do some random steps in the game so you see how the game looks like

rew_tot=0
obs= env.reset()
env.render()
for _ in range(6):
    action = env.action_space.sample() #take step using random action from possible actions (actio_space)
    obs, rew, done, info = env.step(action) 
    rew_tot = rew_tot + rew
    env.render()
    input("Enter")
#Print the reward of these random action
print("Reward: %r" % rew_tot)   

Enter
Enter
Enter
Enter
Enter
Enter
Reward: -3.2025350458327706


In [44]:
discrete = isinstance(env.action_space, gym.spaces.Discrete)
observation_dimention = env.observation_space.shape[0]
action_dimention = env.action_space.n if discrete else env.action_space.shape[0]
print(observation_dimention)
print(action_dimention)

111
8


In [26]:
# action space has 6 possible actions, the meaning of the actions is nice to know for us humans but the neural network will figure it out
print(env.action_space)
NUM_ACTIONS = env.action_space.shape[0]
print(NUM_ACTIONS)

Box(8,)
8


In [30]:
print(env.observation_space)
print()
env.env.s=100 # some random number, you might recognize it
env.render()
input("Enter")
env.env.s = 12 # and some other
env.render()
input("Enter")

Box(111,)

Enter
Enter


''

In [33]:
# Value iteration algorithem
NUM_ACTIONS = env.action_space.shape[0]
print ("NUM_ACTIONS", NUM_ACTIONS)
NUM_STATES = env.observation_space.shape[0]
print ("NUM_STATES", NUM_STATES)
V = np.zeros([NUM_STATES]) # The Value for each state
Pi = np.zeros([NUM_STATES], dtype=int)  # Our policy with we keep updating to get the optimal policy
gamma = 0.9 # discount factor
significant_improvement = 0.01

def best_action_value(s):
    # finds the highest value action (max_a) in state s
    best_a = None
    best_value = float('-inf')

    # loop through all possible actions to find the best current action
    for a in range (0, NUM_ACTIONS):
        env.env.s = s
        s_new, rew, done, info = env.step(a) #take the action
        v = rew + gamma * V[s_new]
        if v > best_value:
            best_value = v
            best_a = a
    return best_a

iteration = 0
while True:
    # biggest_change is referred to by the mathematical symbol delta in equations
    biggest_change = 0
    for s in range (0, NUM_STATES):
        old_v = V[s]
        action = best_action_value(s) #choosing an action with the highest future reward
        env.env.s = s # goto the state
        s_new, rew, done, info = env.step(action) #take the action
        V[s] = rew + gamma * V[s_new] #Update Value for the state using Bellman equation
        Pi[s] = action
        biggest_change = max(biggest_change, np.abs(old_v - V[s]))
    iteration += 1
    if biggest_change < significant_improvement:
        print (iteration,' iterations done')
        break

NUM_ACTIONS 8
NUM_STATES 111


IndexError: arrays used as indices must be of integer (or boolean) type